In [1]:
import pandas as pd
import json
import re

In [5]:
TENSES_LIST = {
"a1_be_have_do_in_the_past": 23,
"a1_can": 14,
"a1_comparative_exept": 4,
"a1_comparative_long": 3,
"a1_comparative_short": 2,
"a1_future_simple": 20,
"a1_have_has_got": 22,
"a1_past_simple_irreg": 19,
"a1_past_simple_reg": 18,
"a1_possesive_s_sing": 0,
"a1_possessive_s_plurar": 1,
"a1_present_continuous_act_rn": 17,
"a1_present_simple_3d_pers": 16,
"a1_present_simple_reg_act": 15,
"a1_special_questions": 24,
"a1_superlative_exept": 7,
"a1_superlative_long": 6,
"a1_superlative_short": 5,
"a1_there_is_am_are": 12,
"a1_there_was_were": 11,
"a1_there_will_be": 13,
"a1_to_be_future_will_be": 10,
"a1_to_be_past_was_were": 8,
"a1_to_be_present_is_am_are": 9,
"a1_want_would_like_to": 21,
"a2_adjectives_ed": 46,
"a2_adjectives_ing": 45,
"a2_ask_reported_verb": 57,
"a2_could": 25,
"a2_first_conditional_if_m": 66,
"a2_first_conditional_m_if": 67,
"a2_had_to": 31,
"a2_have_to": 30,
"a2_imp_reported_speech": 54,
"a2_imperatives": 35,
"a2_ing_non_finite_forms": 50,
"a2_is_able_to": 27,
"a2_is_am_are_going_to": 37,
"a2_much_many_a_lot": 43,
"a2_must": 29,
"a2_narrative_tenses": 40,
"a2_passive_modal_verbs": 63,
"a2_passive_past_simple": 61,
"a2_passive_present_cont": 59,
"a2_passive_present_perf": 60,
"a2_passive_present_simple": 58,
"a2_passive_will": 62,
"a2_past_continuous": 39,
"a2_past_perfect_basic": 42,
"a2_past_reported_speech": 53,
"a2_pr_reported_speech": 52,
"a2_prepositions_of_time": 34,
"a2_present_perfect": 41,
"a2_relative_clause_others": 47,
"a2_relative_clause_who_which_that": 48,
"a2_say_reported_verb": 55,
"a2_second_conditional_if_m": 68,
"a2_second_conditional_m_if": 69,
"a2_should": 33,
"a2_tell_reported_verb": 56,
"a2_to_non_finite_forms": 49,
"a2_was_were_be_able_to": 26,
"a2_was_were_going_to": 36,
"a2_will_be_able_to": 28,
"a2_will_be_going_to": 38,
"a2_will_have_to": 32,
"a2_zero_article": 44,
"a2_zero_conditional_if_m": 64,
"a2_zero_conditional_m_if": 65,
"a2_zero_non_finite_forms": 51,
}

In [10]:
def convert_dataset(dataset):
    new_dataset = dataset
    new_dataset = new_dataset[['Labeled Data', 'Label', 'View Label']]
    new_dataset['labels'] = 0
    for i in range(len(new_dataset)):
        answers = json.loads(new_dataset['Label'][i])
        answers = answers.get('classifications', 0)
        if answers != 0:
            # answers = answers]
            if len(answers) > 0:
                answers = answers[0]['answers']
                labels = [item['value'] for item in answers]
                new_dataset['labels'][i] = labels
                # print(labels)
    new_dataset = new_dataset[['Labeled Data', 'labels', 'View Label']]
    new_dataset = new_dataset[new_dataset['labels'] != 0 & (new_dataset['Labeled Data'].duplicated() == False)]
    return new_dataset

In [26]:
def pretty_text(dirty_text):
    bad_chars = {
        "’":"'",
        "‘": "'",
        "“": ' ',
        "”": ' ',
        "—": "-",
        "…": "...",
        "–": "-",
        '"': " ",
        '[': "",
        "]": "",
        '(Laughter)': ' ',
        '(Applause)': ' ',
        '--': '-',
        "&gt;": ''
    }
    new_text = str(dirty_text).strip()
    for bad_char in bad_chars:
        new_text = new_text.replace(bad_char, bad_chars[bad_char])
                
    _replace_whitespace_ = re.compile(r"\s+")
    new_text = _replace_whitespace_.sub(" ", new_text).strip()

    _replace_multiple_whitespaces = re.compile(r'\s{1,}')
    new_text = _replace_multiple_whitespaces.sub(" ", new_text).strip()

    return new_text

In [53]:
dataset_1 = pd.read_csv('export-2021-02-04T12_04_07.333Z.csv', encoding='utf-8')
dataset_1 = convert_dataset(dataset_1)
dataset_1[dataset_1['Labeled Data'].duplicated()]

,Labeled Data,labels


In [3]:
# dataset_300 = pd.read_csv('./multi_label_tenses/300.csv', encoding='utf-8')
# dataset_300 = convert_dataset(dataset_300)
# dataset_300

In [9]:
dataset_ner_pred = pd.read_csv("./combine_data/combine_2_21_21_NER/predicted_tenses_2_16_21_NER_1207_balanced_a1_tokenized.csv")
dataset_ner_pred['source'] = 0
dataset_ner_pred['dup'] = 0

dataset_ner_labeled = pd.read_csv("./combine_data/combine_2_21_21_NER/export-2021-02-19T15_38_59.685Z.csv")
dataset_ner_labeled = dataset_ner_labeled[['Labeled Data']]
dataset_ner_labeled = dataset_ner_labeled.rename(columns={'Labeled Data': 'sent'})
dataset_ner_labeled['type'] = ""
dataset_ner_labeled['original_sent'] = ""
dataset_ner_labeled['source'] = 1
dataset_ner_labeled['dup'] = 0

print(len(dataset_ner_pred))

dataset_ner_pred_labeled_comb = pd.concat([dataset_ner_pred, dataset_ner_labeled], ignore_index=True)

for i in range(len(dataset_ner_labeled)):
  sent = dataset_ner_labeled['sent'][i]
  dataset_ner_pred_labeled_comb['dup'][dataset_ner_pred_labeled_comb['sent'] == str(sent)] = 1
dataset_ner_pred_labeled_comb = dataset_ner_pred_labeled_comb[dataset_ner_pred_labeled_comb['dup'] == 0]
dataset_ner_pred_labeled_comb = dataset_ner_pred_labeled_comb[['sent', 'original_sent']]
dataset_ner_pred_labeled_comb.to_csv("./combine_data/combine_2_21_21_NER/dataset_ner_pred_labeled_comb_2_21_21.csv", index=False, encoding='utf-8')

1207


In [55]:
multi_label_tenses = pd.concat([dataset_1])
multi_label_tenses = multi_label_tenses.rename(columns={'Labeled Data': 'sent', 'labels':'type'})
multi_label_tenses.to_csv("syntetic_dataset_labeled_2_4_21_a1_labeled.csv", index=False, encoding='utf-8')


In [3]:
# Statistics

In [59]:
dataset_stat = pd.read_csv("./combine_data/combined_04_02_2021.csv", encoding='utf-8')

In [61]:
dataset_stat.head()

,sent,type
0,I have never once washed an apple from the sto...,"['a2_present_perfect', 'a1_past_simple_irreg',..."
1,"Under the rules of dueling, Dickinson had to r...","['a2_had_to', 'a1_past_simple_irreg', 'a2_narr..."
2,"Usually, when I tell someone, they give me a b...","['a2_first_conditional_if_m', 'a1_present_simp..."
3,"Unfortunately, the names of the developer comm...","['a2_article_the', 'a1_to_be_present_is_am_are..."
4,We recommend you use a developer command promp...,['a1_possessive_pronouns']


In [62]:
TENSES_LIST_NAMES =  list(TENSES_LIST.keys())
# for i in TENSES_LIST_NAMES:
#     print(i)

In [63]:
dataset_stat[list(TENSES_LIST.keys())] = 0

In [65]:
# dataset_stat.columns

In [113]:
# dataset_1200['a2_article_a_an']
# TENSES_LIST_NAMES

In [66]:
def conv_dataset(dataset):
    new_dataset = dataset
    for i in range(len(dataset)): 
        sent_types = str(dataset['type'][i])
        sent_types = json.loads(sent_types.replace("'",'"'))
        # print(sent_types)
        # sent_types = [TENSES_LIST.get(typ) for typ in sent_types if TENSES_LIST.get(typ, -1) != -1]
        # print(sent_types)
        if len(sent_types) != 0:
            for label in sent_types:
                if label in TENSES_LIST_NAMES:
                    new_dataset[label][i] = 1
        
    return new_dataset

In [67]:
dataset_stat = conv_dataset(dataset_stat)

In [93]:
# dataset_1200.iloc[0]

In [95]:
dataset_1200.to_csv("1200_dataframe_1_29_20.csv", encoding='utf-8')

In [71]:
total = 0
minor_classes = []
for label in TENSES_LIST_NAMES:
    lenght = len(dataset_stat[dataset_stat[label] == 1])
    # print(f"{label} --- {lenght}")
    if lenght < 10:
        print(label)
        minor_classes.append(label)
        total += 1
print(total)

a2_adjectives_ing
a2_ask_reported_verb
a2_first_conditional_m_if
a2_have_to
a2_imp_reported_speech
a2_is_able_to
a2_must
a2_passive_present_cont
a2_passive_present_perf
a2_past_reported_speech
a2_pr_reported_speech
a2_prepositions_of_time
a2_second_conditional_if_m
a2_second_conditional_m_if
a2_tell_reported_verb
a2_was_were_be_able_to
a2_was_were_going_to
a2_will_be_able_to
a2_will_be_going_to
a2_will_have_to
20


In [107]:
# sample = dataset_1200[dataset_1200['a2_passive_modal_verbs'] == 1]['sent'].sample(frac=1)[:20]
# sa
# for i in range(len(sample)):
#     print(sample.iloc[i])

While I found Weather Channel's Bus Shelter prank amusing, others might be frustrated for getting wet in what they will later regard as a stupid prank.
It has to do with a broad, consistent logic, and that logic could be understood as one occupies the building.
Devices that can be made in PolyMUMPs include: Acoustics(microphones), Sensors, Accelerometers, Micro-fluidics, and Display Technologies.
We want to have a community where you can upload ideas, and those ideas can be tested in an earthquake, in flood, in all sorts of austere environments.
Time blocking can be wise if you use it appropriately and if you ease into it.
It is a beautiful and terrible thing, and should therefore be treated with great caution.
Tight schedules with no flexibility can be overwhelming and feeling like you didn't accomplish a certain goal can be taxing to your mental health.
However, mild competition can be healthy and in fact, a necessary source of inspiration.
The study acknowledges that the anxiety tha

In [118]:
syntetic_dataset = pd.DataFrame(columns=minor_classes)
syntetic_dataset.to_csv("syntetic_dataset.csv", index=False)

# Check synthetic dataset

In [39]:
syntetic_dataset_labeled_2_4_21_a1 = pd.read_csv("syntetic_dataset_labeled_2_4_21_a1.csv", encoding='utf-8')
syntetic_dataset_labeled_2_4_21_a1_transformed = pd.DataFrame(columns=['sent', 'type'])

for column in list(syntetic_dataset_labeled_2_4_21_a1.columns):
    column_sents = syntetic_dataset_labeled_2_4_21_a1[column]
    for sent in column_sents:
        row = {}
        row['sent'] = pretty_text(str(sent))
        row['type'] = str(column)
        syntetic_dataset_labeled_2_4_21_a1_transformed = syntetic_dataset_labeled_2_4_21_a1_transformed.append(row, ignore_index=True)

syntetic_dataset_labeled_2_4_21_a1_transformed = syntetic_dataset_labeled_2_4_21_a1_transformed[syntetic_dataset_labeled_2_4_21_a1_transformed['sent'].duplicated() == False]
syntetic_dataset_labeled_2_4_21_a1_transformed.to_csv("syntetic_dataset_labeled_2_4_21_a1_transformed.csv", encoding='utf-8', index=False)

### Check dublicates

In [40]:
syntetic_dataset_labeled_2_4_21_a1_transformed = pd.read_csv('syntetic_dataset_labeled_2_4_21_a1_transformed.csv')
syntetic_dataset_labeled_2_4_21_a1_transformed[syntetic_dataset_labeled_2_4_21_a1_transformed['sent'].duplicated()]

,sent,type


### Check on previous dataset

In [47]:
prev_dataset = pd.read_csv("./combine_data/combined_29_01_2021.csv")
current_dataset = pd.read_csv("syntetic_dataset_labeled_2_4_21_a1_transformed.csv")
prev_current_dataset = pd.concat([prev_dataset, current_dataset])
prev_current_dataset[prev_current_dataset.duplicated()]
temp = prev_current_dataset
temp.to_csv('temp.csv', index=False)